<a href="https://colab.research.google.com/github/BuberDev/Artificial-Intelligence-ComputerVision/blob/main/Klasyfikator_binarny_Drone_vs_Passenger_Plane.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Import bibliotek

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import shutil

import plotly.graph_objects as go
from sklearn.metrics import confusion_matrix, classification_report

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

np.set_printoptions(precision=12, suppress=True, linewidth=120)


## 2. Zaladowanie danych

In [2]:
#!wget https://storage.googleapis.com/esmartdata-courses-files/ann-course/flying-vehicles.zip

--2023-06-28 20:53:37--  https://storage.googleapis.com/esmartdata-courses-files/ann-course/flying-vehicles.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 142.251.161.128, 142.251.172.128, 74.125.126.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|142.251.161.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1540232238 (1.4G) [application/x-zip-compressed]
Saving to: ‘flying-vehicles.zip’

flying-vehicles.zip 100%[===================>]   1.43G  32.5MB/s    in 46s     

2023-06-28 20:54:24 (31.7 MB/s) - ‘flying-vehicles.zip’ saved [1540232238/1540232238]



In [3]:
#!unzip -q flying-vehicles.zip

Przygotowanie zbiorow: treningowego, walidacyjnego i testowego

In [4]:
base_dir = './data/planes'
raw_no_of_files = {}
classes = ['drone', 'fighter-jet', 'helicopter', 'missile', 'passenger-plane', 'rocket']

# Ponizszy fragment kodu sprawdzi nam ile mamy zdiec dla poszczegolnych klass (tych wyzej)
for dir in classes:
  raw_no_of_files[dir] = len(os.listdir(os.path.join(base_dir, dir)))

raw_no_of_files.items()

dict_items([('drone', 1332), ('fighter-jet', 1867), ('helicopter', 1387), ('missile', 1334), ('passenger-plane', 1424), ('rocket', 1186)])

In [6]:
from IPython.lib.display import exists
# Torzymy katalog images
data_dir = './images'

# Jesli taki katelolg nie istnieje to tworzymy go
if not os.path.exists( data_dir):
  os.mkdir(data_dir)

# Nastepnie utworzymy sobie odpowiednio sciezki do katalogow gdzie bedziemy miec zbory trenigowy, wal i test
# Np teraz dzieki metodzie 'join' polaczymy sciezke katalogu 'data_dir' z nowo utworzonym katalogiem 'train'

train_dir = os.path.join(data_dir, 'train')
valid_dir = os.path.join(data_dir, 'valid')
test_dir = os.path.join(data_dir, 'test')

# Teraz tworzemy kolejny poziom zagniezdzenia katalogow. Tym razem dla dwoch klass (klasyfikator binarny). Drony i samoloty pasazerskie

train_drone_dir = os.path.join(train_dir, 'drone')
train_passenger_dir = os.path.join(train_dir, 'passenger')

valid_drone_dir = os.path.join(valid_dir, 'drone')
valid_passenger_dir = os.path.join(valid_dir, 'passenger')

test_drone_dir = os.path.join(test_dir, 'drone')
test_passenger_dir = os.path.join(test_dir, 'passenger')

# Tworzymy teraz petle ktora przejdzie nam po 3 katalogach z 2 poziomu i jesli jakis nie istnieje to utworzy go
for directory in (train_dir, valid_dir, test_dir):
  if not os.path.exists(directory):
    os.mkdir(directory)

# To samo robimy dla tych kolejnych katalogow z 3 poziomu
dirs = [train_drone_dir, train_passenger_dir, valid_drone_dir, valid_passenger_dir, test_drone_dir, test_passenger_dir]

for dir in dirs:
  if not os.path.exists(dir):
    os.mkdir(dir)


In [ ]:
# Teraz musimy stworzyc pelne sciezki do naszych plikow po niewaz chcemy je przekopiowac
# Do zmiennej 'drone_filenames' przypiszemy sobie wylistowanie plikow z katalodu 'drone'

drone_filenames = os.listdir(os.path.join(base_dir, 'drone'))
passenger_filenames = os.listdir(os.path.join(base_dir, 'passenger-plane'))

# Teraz przeprowadzimy proces , w ktorym wybierzemy z tych katalogow tylko te pliki ktorych koncowki/rozszerzenie konczy sie na 'jpg', 'png', 'jpeg'
